In [66]:
import numpy as np
import pandas as pd

df1 = pd.read_excel('2021W43 Input.xlsx', sheet6_name=0)
df2 = pd.read_excel('2021W43 Input.xlsx', sheet_name=1, skiprows=5)
df3 = pd.read_excel('2021W43 Input.xlsx', sheet_name=2)

df1['Rating'] = df1['Rating'].astype(int)
df1 = pd.merge(df1,df3,how='inner',left_on='Rating',right_on='Risk level')

df1['Date lodged'] = df1['Year'].astype(str) + df1['Month '].astype(str).str.zfill(2) + df1['Date'].astype(str).str.zfill(2)
df1['Date lodged'] = pd.to_datetime(df1['Date lodged'], format="%Y/%m/%d")

df1 = df1.drop(columns=['Year', 'Month ', 'Date', 'Risk level', 'Rating'])
df1 = df1.rename(columns={'Business Unit ':'Unit', 'Risk rating':'Rating'})

In [57]:
df1.head()

,Ticket ID,Unit,Owner,Issue,Management Strategy,Status,Rating,Date lodged
0,1,A,Jimmy Collins,Item was the wrong colour,Phosfluorescently engage worldwide methodologi...,In Progress,Medium,2021-04-17
1,11,A,Terry Sheppard,Sold the wrong product,Leverage agile frameworks to provide a robust ...,Completed,Medium,2021-12-15
2,12,A,Terry Sheppard,Item didn't fit,Bring to the table win-win survival strategies...,Completed,Medium,2021-05-21
3,13,A,Terry Sheppard,Item was the wrong colour,Capitalize on low hanging fruit to identify a ...,Completed,Medium,2021-11-20
4,17,A,Terry Sheppard,Item was the wrong colour,Phosfluorescently expedite impactful supply ch...,In Progress,Medium,2021-07-13


In [68]:
df2['Date lodged'] = pd.to_datetime(df2['Date lodged'], format="%d/%m/%Y")
df2.head()

,Ticket ID,Unit,Owner,Issue,Management Strategy,Date lodged,Status,Rating
0,19,B,Garret Hatfield,Sold the wrong product,Objectively innovate empowered manufactured pr...,2021-09-01,In Progress,Low
1,20,B,Amanda Williams,Item didn't fit,Proactively envisioned multimedia based expert...,2021-10-03,In Progress,Low
2,21,B,Jimmy Collins,Item was the wrong colour,"Credibly innovate granular internal or ""organi...",2021-02-02,Completed,Medium
3,22,B,Amanda Williams,Delivery delayed,Interactively procrastinate high-payoff conten...,2021-02-06,In Progress,High
4,23,B,Jimmy Collins,Delivery delayed,Globally incubate standards compliant channels...,2021-10-16,Completed,Medium


In [83]:
unit_df = pd.concat([df1,df2])
unit_df.head()

,Ticket ID,Unit,Owner,Issue,Management Strategy,Status,Rating,Date lodged
0,1,A,Jimmy Collins,Item was the wrong colour,Phosfluorescently engage worldwide methodologi...,In Progress,Medium,2021-04-17
1,11,A,Terry Sheppard,Sold the wrong product,Leverage agile frameworks to provide a robust ...,Completed,Medium,2021-12-15
2,12,A,Terry Sheppard,Item didn't fit,Bring to the table win-win survival strategies...,Completed,Medium,2021-05-21
3,13,A,Terry Sheppard,Item was the wrong colour,Capitalize on low hanging fruit to identify a ...,Completed,Medium,2021-11-20
4,17,A,Terry Sheppard,Item was the wrong colour,Phosfluorescently expedite impactful supply ch...,In Progress,Medium,2021-07-13


In [84]:
unit_df['classification'] = np.where(unit_df['Date lodged'] < '2021-10-01', 'Opening cases', 'New cases')

In [85]:
unit_pivot = unit_df.melt(id_vars=['Ticket ID', 'Rating'], value_vars=['Status', 'classification'], value_name='Values')

In [73]:
unit_pivot

,Ticket ID,Rating,variable,Values
0,1,Medium,Status,In Progress
1,11,Medium,Status,Completed
2,12,Medium,Status,Completed
3,13,Medium,Status,Completed
4,17,Medium,Status,In Progress
...,...,...,...,...
63,30,Medium,classification,Opening cases
64,31,Medium,classification,Opening cases
65,32,Medium,classification,New cases
66,33,Low,classification,Opening cases


In [86]:
unit_pivot = unit_pivot.pivot_table(values='Ticket ID', index='Rating', columns=['Values'], aggfunc='size',fill_value=0)
unit_pivot

Values,Completed,Deferred,In Progress,New cases,Opening cases
Rating,,,,,
High,1,1,5,1,6
Low,4,0,8,3,9
Medium,8,2,5,7,8


In [75]:
unit_pivot = unit_pivot.drop(columns=['In Progress']).reset_index()
unit_pivot

Values,Rating,Completed,Deferred,New cases,Opening cases
0,High,1,1,1,6
1,Low,4,0,3,9
2,Medium,8,2,7,8


In [77]:
unit_pivot['Continuing'] = unit_pivot['Opening cases'] + unit_pivot['New cases'] - unit_pivot['Completed'] - unit_pivot['Deferred']
unit_pivot

Values,Rating,Completed,Deferred,New cases,Opening cases,Continuing
0,High,1,1,1,6,5
1,Low,4,0,3,9,8
2,Medium,8,2,7,8,5


In [78]:
output_df = unit_pivot.melt(id_vars=['Rating'], var_name='Status', value_name='Cases')
output_df

,Rating,Status,Cases
0,High,Completed,1
1,Low,Completed,4
2,Medium,Completed,8
3,High,Deferred,1
4,Low,Deferred,0
5,Medium,Deferred,2
6,High,New cases,1
7,Low,New cases,3
8,Medium,New cases,7
9,High,Opening cases,6
